In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from scipy import ndimage 
tf.compat.v1.set_random_seed(1)

In [ ]:
# ------------------------------------------- Utility Functions -------------------------------------------
""" Creates subsequences of the original sequence to fit LSTM structure
 
Args:
    sequence_1: the first sequence which gets converted into multiple subarrays of length: n_steps
    sequence_2: the second sequence, each n_steps'th element will be part of the output array
    n_steps: the amount of time steps used as an input into the LSTM for prediction

Returns:
    A tuple of 2 numpy arrays in the required format
    
    X.shape = (X.shape[0] - n_steps, n_steps)
    y.shape = (X.shape[0] - n_steps, 1)

"""
def subsequences(sequence_X, sequence_y, n_steps):
    if n_steps > len(sequence_X):
        raise Exception('subsequences: n_steps should not exceed the sequence length')
    
    X, y = list(), list()
    for i in range(len(sequence_X)):
        end_ix = i + n_steps

        if end_ix > len(sequence_X):
            break

        X.append(sequence_X[i:end_ix])
        y.append(sequence_y[end_ix-1])
        
    return np.array(X), np.array(y)


""" Subsample array to decrease the amount of data

Args:
    sequence: the input array to be subsampled
    d_sample: sample frequency, meaning every d_sample'th element will be part of the output
    
Returns:
    The subsampled array

"""
def subsample(sequence, d_sample):
    return sequence[::d_sample]


""" Smooth array to decrease measurement noise

Args: 
    sequence: the input array to be smoothed
    sigma: parameter for the gauss filtering

Returns:
    The smoothed array
"""
def smooth(sequence, sigma):
    return ndimage.filters.gaussian_filter(sequence, sigma)


""" Aligns two sequences

    In this context this means subsampling the first array so that it afterwards has the same size as the second array
    
Args: 
    sequence_1: arrray to be aligned
    sequence_2: array to be aligned to
    
Returns:
    The algined array
"""
def align(sequence_1, sequence_2):
    if len(sequence_1) < len(sequence_2):
        raise Exception('align: missmatch of sequence lengths')
    
    sample_ratio = sequence_1.shape[0] / sequence_2.shape[0]

    aligned_sequence = list()
    for i in range(len(sequence_2)):
        aligned_sequence.append(sequence_1[int(np.round(i * sample_ratio))])

    aligned_sequence = np.array(aligned_sequence)
    
    return aligned_sequence


""" Prepares the data for input into the LSTM

    Preparation incudes:
    subsampling, smoothing, aligning differnt sized sequences and reshaping the sequence to the requested format
    
Args:
    input_sequence: the input feature sequence
    label_sequence: the output/groud truth sequence
    aligned: indicates if input and label sequence are of equal size or need alignment
    d_sample: sample frequency
    n_steps: the amount of time steps used as an input into the LSTM for prediction
    sigma: parameter for the data smoothing

Returns:
    A tuple of 3 values. The prepared input sequence X, the output sequence of labels y and the scaler component for y. 
    This is needed afterwards to scale the output back to the original value range
"""
def prepare_data(input_sequence, label_sequence, aligned, d_sample, n_steps, sigma):
    # align data if not of equal size
    if not aligned:        
        input_sequence = align(input_sequence, label_sequence)

    # subsample and smooth data 
    input_sequence_ = subsample(input_sequence, d_sample)
    input_sequence_ = smooth(input_sequence_, sigma)
    
    label_sequence_ = subsample(label_sequence, d_sample)
    label_sequence_ = smooth(label_sequence_, sigma)

    # convert into X and y sequences
    X, y = subsequences(input_sequence_, label_sequence_, n_steps)
    y = np.reshape(y, (-1, 1))

    # fit and scale X
    scaler_X = MinMaxScaler(feature_range = (0, 1))
    scaler_X.fit(X)
    X_scaled = scaler_X.transform(X)

    # fit and scale y
    scaler_y = MinMaxScaler(feature_range = (0, 1))
    scaler_y.fit(y)
    y_scaled = scaler_y.transform(y)

    # reshape into correct format
    X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)
    
    return X_scaled, y_scaled, scaler_y

def prepare_data_unscaled(input_sequence, label_sequence, aligned, d_sample, n_steps, sigma):
    # align data if not of equal size
    if not aligned:        
        input_sequence = align(input_sequence, label_sequence)

    # subsample and smooth data 
    input_sequence_ = subsample(input_sequence, d_sample)
    input_sequence_ = smooth(input_sequence_, gauss_sigma)
    
    label_sequence_ = subsample(label_sequence, d_sample)
    label_sequence_ = smooth(label_sequence_, gauss_sigma)

    # convert into X and y sequences
    X, y = subsequences(input_sequence_, label_sequence_, n_steps)
    y = np.reshape(y, (-1, 1))

    # reshape into correct format
    X = X.reshape(X.shape[0], X.shape[1], 1)
    
    return X, y

In [ ]:
# ------------------------------------------- Hyperparameters -------------------------------------------
n_steps = 50
n_features = 2
n_lstm_units_1 = 50
n_lstm_units_2 = 20
n_dense_units = 10
n_epochs = 10

d_sample = 1
gauss_sigma = 10

In [ ]:
# ------------------------------------------- Prepare Training Data -------------------------------------------
# single profile
train_cur_inv = np.loadtxt('../data/fobss_data/data/osc_19_11_18/inverter/Inverter_Current.csv', delimiter=';')
train_cur_inv = train_cur_inv[:,1]
train_volt_slave_0_cell_4 = np.loadtxt('../data/fobss_data/data/osc_19_11_18/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
train_volt_slave_0_cell_4 = train_volt_slave_0_cell_4[:,4]
train_volt_repeat = np.full(shape=train_volt_slave_0_cell_4.shape[0], fill_value=train_volt_slave_0_cell_4[0], dtype=np.float)

X1_train, y_train_single, _ = prepare_data(train_cur_inv, train_volt_slave_0_cell_4, False, d_sample, n_steps, gauss_sigma)

inv_cum = np.cumsum(train_cur_inv)
X2_train, _, _ = prepare_data(inv_cum, train_volt_slave_0_cell_4, False, d_sample, n_steps, gauss_sigma)

X_train_single = np.append(X1_train, X2_train, axis=2)

print('Single Profile:', X_train_single.shape, y_train_single.shape)

# multiple profiles 
cur_profile_1 = np.loadtxt('../data/fobss_data/data/Profile 10A/inverter/Inverter_Current.csv', delimiter=';')
cur_profile_1 = cur_profile_1[:,1]
volt_profil_1 = np.loadtxt('../data/fobss_data/data/Profile 10A/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
volt_profil_1 = volt_profil_1[:,4]

cur_profile_2 = np.loadtxt('../data/fobss_data/data/Profile -10A/inverter/Inverter_Current.csv', delimiter=';')
cur_profile_2 = cur_profile_2[:,1]
volt_profil_2 = np.loadtxt('../data/fobss_data/data/Profile -10A/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
volt_profil_2 = volt_profil_2[:,4]

cur_profile_3 = np.loadtxt('../data/fobss_data/data/Profile 25A/inverter/Inverter_Current.csv', delimiter=';')
cur_profile_3 = cur_profile_3[:,1]
volt_profil_3 = np.loadtxt('../data/fobss_data/data/Profile 25A/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
volt_profil_3 = volt_profil_3[:,4]

cur_profile_4 = np.loadtxt('../data/fobss_data/data/Profile -25A/inverter/Inverter_Current.csv', delimiter=';')
cur_profile_4 = cur_profile_4[:,1]
volt_profil_4 = np.loadtxt('../data/fobss_data/data/Profile -25A/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
volt_profil_4 = volt_profil_4[:,4]

# align and cut multiple profiles
cur_profile_1 = align(cur_profile_1, cur_profile_1)
cur_profile_1 = cur_profile_1[:2000]
volt_profile_1 = volt_profil_1[:2000]

cur_profile_2 = align(cur_profile_2, cur_profile_2)
cur_profile_2 = cur_profile_2[:2000]
volt_profile_2 = volt_profil_2[:2000]

cur_profile_3 = align(cur_profile_3, cur_profile_3)
cur_profile_3 = cur_profile_3[:2000]
volt_profile_3 = volt_profil_3[:2000]

cur_profile_4 = align(cur_profile_4, cur_profile_4)
cur_profile_4 = cur_profile_4[:2000]
volt_profile_4 = volt_profil_4[:2000]

# shape into correct input format
stacked_prof_volt = np.stack([volt_profile_1, volt_profile_2, volt_profile_3, volt_profile_4], axis=0).flatten()
stacked_prof_cur = np.stack([cur_profile_1, cur_profile_2, cur_profile_3, cur_profile_4], axis=0).flatten()

X1_train, y_train, _ = prepare_data(stacked_prof_cur, stacked_prof_volt, False, d_sample, n_steps, gauss_sigma)

# prepare second input data
stacked_prof_volt = np.stack([volt_profile_1, volt_profile_2, volt_profile_3, volt_profile_4], axis=0).flatten()
stacked_prof_cur = np.stack([cur_profile_1, cur_profile_2, cur_profile_3, cur_profile_4], axis=0).flatten()
cum_prof_cur = np.cumsum(stacked_prof_cur)
X2_train, _, _ = prepare_data(cum_prof_cur, stacked_prof_volt, False, d_sample, n_steps, gauss_sigma)

X_train = np.append(X1_train, X2_train, axis=2)
print('Multiple Profiles:', X_train.shape, y_train.shape)

In [ ]:
# ------------------------------------------- Initialize LSTM -------------------------------------------
model = keras.Sequential()

# Adding the first LSTM layer and some Dropout regularisation
model.add(layers.LSTM(units = n_lstm_units_1, input_shape = (n_steps, n_features), return_sequences=True))
model.add(layers.LeakyReLU(alpha=0.1))

model.add(layers.LSTM(units = n_lstm_units_2))
model.add(layers.LeakyReLU(alpha=0.1))
# Adding the output layer
model.add(layers.Dense(1, activation='tanh'))

# Show model
model.summary()

In [ ]:
# ------------------------------------------- Train LSTM -------------------------------------------
model.compile(optimizer = 'adam', loss = 'mse', metrics=["mae"])

# Fitting the LSTM to the Training set
history = model.fit(X_train_single, y_train_single, epochs = n_epochs, verbose = 1)

In [ ]:
# ------------------------------------------- Visualize Training -------------------------------------------
loss = history.history['loss']
metrics = history.history['mae']
epochs = range(1,len(loss)+1)

# plot graph
plt.subplots(figsize = (5,5))
plt.subplot(2,1,1)
plt.plot(epochs,loss,'-o',label='training loss')
plt.legend()
plt.subplot(2,1,2)
plt.plot(epochs,metrics,'-o', color='green',label='absolute error')
plt.legend()

In [ ]:
# ------------------------------------------- Prepare Test Data -------------------------------------------
# load data
test_cur_inv = np.loadtxt('../data/fobss_data/data/osc_19_11_18/inverter/Inverter_Current.csv', delimiter=';')
test_cur_inv = test_cur_inv[:,1]
test_volt_slave_0_cell_4 = np.loadtxt('../data/fobss_data/data/osc_19_11_18/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
test_volt_slave_0_cell_4 = test_volt_slave_0_cell_4[:,4]

# prepare prediction
X1_test1, y_test1, scaler_y1 = prepare_data(test_cur_inv, test_volt_slave_0_cell_4, False, d_sample, n_steps, gauss_sigma)
inv_cum1 = np.cumsum(test_cur_inv)
X2_test1, _, _ = prepare_data(inv_cum1, test_volt_slave_0_cell_4, False, d_sample, n_steps, gauss_sigma)

X_test1 = np.append(X1_test1, X2_test1, axis=2)

print(X_test1.shape, y_test1.shape)

# load data
test_cur_inv = np.loadtxt('../data/fobss_data/data/stairs_19_11_18/inverter/Inverter_Current.csv', delimiter=';')
test_cur_inv = test_cur_inv[:,1]
test_volt_slave_0_cell_4 = np.loadtxt('../data/fobss_data/data/stairs_19_11_18/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
test_volt_slave_0_cell_4 = test_volt_slave_0_cell_4[:,4]

# prepare prediction
X1_test2, y_test2, scaler_y2 = prepare_data(test_cur_inv, test_volt_slave_0_cell_4, False, d_sample, n_steps, gauss_sigma)
inv_cum2 = np.cumsum(test_cur_inv)
X2_test2, _, _ = prepare_data(inv_cum2, test_volt_slave_0_cell_4, False, d_sample, n_steps, gauss_sigma)

X_test2 = np.append(X1_test2, X2_test2, axis=2)

print(X_test2.shape, y_test2.shape)

# load data
test_cur_inv = np.loadtxt('../data/fobss_data/data/Profile 10A 3x/inverter/Inverter_Current.csv', delimiter=';')
test_cur_inv = test_cur_inv[:,1]
test_volt_slave_0_cell_4 = np.loadtxt('../data/fobss_data/data/Profile 10A 3x/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
test_volt_slave_0_cell_4 = test_volt_slave_0_cell_4[:,4]

# prepare prediction
X1_test3, y_test3, scaler_y3 = prepare_data(test_cur_inv, test_volt_slave_0_cell_4, False, d_sample, n_steps, gauss_sigma)
inv_cum3 = np.cumsum(test_cur_inv)
X2_test3, _, _ = prepare_data(inv_cum3, test_volt_slave_0_cell_4, False, d_sample, n_steps, gauss_sigma)

X_test3 = np.append(X1_test3, X2_test3, axis=2)

print(X_test3.shape, y_test3.shape)

In [ ]:
# ------------------------------------------- Predict on Test Data -------------------------------------------
yhat1 = model.predict(X_test1, verbose = 1)
yhat_rescaled1 = scaler_y1.inverse_transform(yhat1)
y_test_unscaled1 = scaler_y1.inverse_transform(y_test1)

yhat2 = model.predict(X_test2, verbose = 1)
yhat_rescaled2 = scaler_y2.inverse_transform(yhat2)
y_test_unscaled2 = scaler_y2.inverse_transform(y_test2)

yhat3 = model.predict(X_test3, verbose = 1)
yhat_rescaled3 = scaler_y3.inverse_transform(yhat3)
y_test_unscaled3 = scaler_y3.inverse_transform(y_test3)


# plot test results
plt.subplots(figsize = (7,10))
plt.subplot(3,1,1)
plt.plot(yhat_rescaled1, color='red', label = 'predicted')
plt.plot(y_test_unscaled1, color='blue', label = 'measured')
plt.legend()
plt.subplot(3,1,2)
plt.plot(yhat_rescaled2, color='red', label = 'predicted')
plt.plot(y_test_unscaled2, color='blue', label = 'measured')
plt.legend()
plt.subplot(3,1,3)
plt.plot(yhat_rescaled3, color='red', label = 'predicted')
plt.plot(y_test_unscaled3, color='blue', label = 'measured')
plt.legend()
plt.show()